# Relatório de Data Quality de APIs Financeiras

Este notebook tem como objetivo coletar dados de diferentes APIs financeiras (como SELIC e câmbio USD/BRL), realizar transformações, calcular métricas relevantes e gerar um relatório de Data Quality para cada fonte.

In [1]:
# Importando bibliotecas necessárias
import os
import pandas as pd
import requests as rq

In [ ]:
# Realizando coleta de dados da API de taxa Selic do Banco Central
url = 'https://api.bcb.gov.br/dados/serie/bcdata.sgs.11/dados'
params = {'dataInicial': '01/01/2025','dataFinal': '22/08/2025'}

res = rq.get(url, params=params, timeout=30)
res.raise_for_status()
dados = res.json()

df = pd.DataFrame(dados)

# Transforma o campo data pra padrão americano yyyy-mm-dd
df["data"] = pd.to_datetime(df["data"], dayfirst=True) # parâmetro dayfirst=True diz que o formato da data está no padrão brasileiro (dd/mm/yyyy)

# Transformando o valor para para tipo float
df["taxa_diaria_percentual"] = df["valor"].astype(float)

# Realizando a multiplicação de 252 (dias de negociação) para dados relacionados ao mercado conforme a fonte https://www.investopedia.com/terms/a/annualize.asp
# Nesse primeiro comando estou dividindo por 100 pra realizar o calculo de exponenciações
df["taxa_diaria_decimal"] = df["taxa_diaria_percentual"] / 100.0

# Anualização por 252 dias úteis conforme a documentação da fonte
df["taxa_selic"] = ((1 + df["taxa_diaria_decimal"]) ** 252 - 1) * 100

# Selecionando somente as colunas data e taxa_selic
df = df[['data', 'taxa_selic']]
df_selic = df


In [3]:
# Realizando a coleta de dados da API Exchangerate.host com dados de cotação do dólar

# Lendo a variavel de ambiente com chave de acesso à API
key = os.getenv('exchangerate_key')

# Definindo parâmetros
url = 'https://api.exchangerate.host/timeframe'
params = {'access_key': key,  'start_date': '2025-01-01', 'end_date': '2025-08-22', 'source': 'USD', 'currencies': 'BRL'}

# Faz a requisição da API e transforma em dicionário
res = rq.get(url, params=params)
res_dict = res.json()

# Criando uma lista de dicionários onde .items() retorna pares de chave e valor do dicionário
# Sendo a coluna date recebe a chave e USDBRL recebe o valor que está dentro do dicionário
# Usando o loop for pra percorrer cada item do dicionário res_dict['quotes']
list_dict = [{'date': date, 'USDBRL': value['USDBRL']} for date, value in res_dict['quotes'].items()]

# Transforma em Dataframe do pandas
df = pd.DataFrame(list_dict)

# Converter a coluna de data para datetime
df['date'] = pd.to_datetime(df['date'])

df_dolar = df

In [6]:
# Função de Data Quality com coluna API como primeira
def relatorio_data_quality(df, col_data, col_valor, fonte):
    relatorio = {
        'API': fonte,
        'total_registros': len(df),
        'data_inicial': df[col_data].min().strftime('%Y-%m-%d'),
        'data_final': df[col_data].max().strftime('%Y-%m-%d'),
        'duplicatas': df[col_data].duplicated().sum(),
        'nulos_data': df[col_data].isna().sum(),
        'nulos_valor': df[col_valor].isna().sum(),
        'datas_faltantes': len(
            set(pd.date_range(df[col_data].min(), df[col_data].max(), freq='B'))
            - set(pd.to_datetime(df[col_data].unique()))
        ),
        'valores_negativos': (df[col_valor] < 0).sum(),
        'valores_acima_50': (df[col_valor] > 50).sum(),
        'min_valor': df[col_valor].min(),
        'max_valor': df[col_valor].max(),
        'media_valor': df[col_valor].mean(),
        'desvio_padrao': df[col_valor].std()
        
    }
    df_rel = pd.DataFrame([relatorio])
    
    return df_rel


In [8]:
# Relatório SELIC
df_dq_selic = relatorio_data_quality(df_selic, 'data', 'taxa_selic', 'API SELIC')

# Relatório USD/BRL
df_dq_cambio = relatorio_data_quality(df_dolar, 'date', 'USDBRL', 'API USD/BRL')

# Junta os dois relatórios
df_dq_final = pd.concat([df_dq_selic, df_dq_cambio], ignore_index=True)

df_dq_final

,API,total_registros,data_inicial,data_final,duplicatas,nulos_data,nulos_valor,datas_faltantes,valores_negativos,valores_acima_50,min_valor,max_valor,media_valor,desvio_padrao
0,API SELIC,160,2025-01-02,2025-08-21,0,0,0,6,0,0,12.149954,14.899981,13.998389,0.945510
1,API USD/BRL,234,2025-01-01,2025-08-22,0,0,0,0,0,0,5.392698,6.301501,5.704443,0.191647
